In [1]:
import coremltools
%load_ext autoreload
%autoreload 2

## Load pre-trained keras model

In [2]:
from keras.models import load_model
# model = load_model('./model/nn4.small2.lrn.h5')

import tensorflow as tf
from keras.utils import CustomObjectScope
with CustomObjectScope({'tf': tf}):
    model = load_model('./model/nn4.small2.lrn.h5' )

/Users/victor_sy_wang/anaconda/lib/python2.7/site-packages/keras/models.py:245: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '


In [3]:
import coremltools
coreml_model = coremltools.converters.keras.convert(
  model, input_names='data', image_input_names='data', image_scale=1/255.0, output_names='output')

('fuck input_dims', [(96, 96, 3)])
('fuck undiltered_shape', [(None, 96, 96, 3)])
fuck reverse
('fuck modified input_dims', [(3, 96, 96)])
('fuck modified output_dims', [[128]])
0 : input_6, <keras.engine.topology.InputLayer object at 0x125c1ee10>
1 : zero_padding2d_97, <keras.layers.convolutional.ZeroPadding2D object at 0x125d596d0>
2 : conv1, <keras.layers.convolutional.Conv2D object at 0x125d591d0>
3 : bn1, <keras.layers.normalization.BatchNormalization object at 0x125c1ed90>
4 : activation_152, <keras.layers.core.Activation object at 0x125d98290>
5 : zero_padding2d_98, <keras.layers.convolutional.ZeroPadding2D object at 0x1110d4b50>
6 : max_pooling2d_26, <keras.layers.pooling.MaxPooling2D object at 0x125e7abd0>
7 : lrn_1, <keras.layers.core.Lambda object at 0x125c1ecd0>
('fuck lambda', u'lrn_1')
yes lrn
8 : conv2, <keras.layers.convolutional.Conv2D object at 0x125d59450>
9 : bn2, <keras.layers.normalization.BatchNormalization object at 0x125e9a590>
10 : activation_153, <keras.layer

In [4]:
print coreml_model

input {
  name: "data"
  type {
    imageType {
      width: 96
      height: 96
      colorSpace: RGB
    }
  }
}
output {
  name: "output"
  type {
    multiArrayType {
      shape: 128
      dataType: DOUBLE
    }
  }
}



In [5]:
# Read a sample image as input to test the model
import cv2
import numpy as np
img = cv2.imread('./data/dlib-affine-sz/Aaron_Eckhart/Aaron_Eckhart_0001.png', 1)
img = img[...,::-1]
img = np.around(np.transpose(img, (2, 1, 0))/255.0, decimals=12)

# x_train = np.array(img)
# y = coreml_model.predict({'image': x_train})

img = np.transpose(img, (1, 2, 0))
x_train = np.array([img])
y = model.predict_on_batch(x_train)

print y

[[ 0.09772716  0.02655629  0.00130884 -0.05884866  0.12626688  0.11332991
   0.15292448  0.05099031 -0.17987581  0.10544605 -0.04671881  0.06534135
   0.15785125 -0.00186696  0.15121728 -0.09237478  0.17005576  0.06216982
  -0.00902624  0.11714683  0.04019579 -0.01088192  0.02814879  0.13169518
  -0.05595746  0.00503826 -0.15796292 -0.12085992  0.117665    0.07790123
  -0.0062046   0.13924114 -0.03397679  0.09833062  0.05024977 -0.03000071
   0.06631183  0.17160252 -0.01937551 -0.12199549  0.0514313   0.00463624
   0.00581299  0.02534417 -0.20161469  0.03190511  0.08420721  0.06971633
  -0.09853088  0.00507045  0.05463877 -0.02829943 -0.04770465 -0.0199649
   0.09869562 -0.0003353  -0.04359384  0.1426736  -0.01388688 -0.07596582
  -0.0815493  -0.01652406  0.05792676 -0.18034409  0.12237014 -0.03725079
   0.00660999 -0.00029476 -0.18744253 -0.02574466 -0.00119687  0.04808429
  -0.08070923  0.04007214 -0.05527268  0.01918375  0.0133779   0.04876364
   0.17812441 -0.00252923 -0.10216619  

In [158]:
# LFW TEST
import lfw
import os
import numpy as np
import math
import facenet
import time
import tensorflow as tf

lfw_pairs='data/pairs.txt'
lfw_dir='data/dlib-affine-sz'
lfw_file_ext='png'
lfw_nrof_folds=10
image_size=96
batch_size=100

# Read the file containing the pairs used for testing
pairs = lfw.read_pairs(os.path.expanduser(lfw_pairs))

# Get the paths for the corresponding images
paths, actual_issame = lfw.get_paths(os.path.expanduser(lfw_dir), pairs, lfw_file_ext)

embedding_size=128
nrof_images = len(paths)
nrof_batches = int(math.ceil(1.0*nrof_images / batch_size))
emb_array = np.zeros((nrof_images, embedding_size))

# print paths

for i in range(nrof_batches):
  start_index = i*batch_size
  end_index = min((i+1)*batch_size, nrof_images)
  paths_batch = paths[start_index:end_index]
  images = facenet.load_data(paths_batch, False, False, image_size)
  images = np.transpose(images, (0,3,1,2))
  
  t0 = time.time()
  y = []
  for img in images:
    tmp = coreml_model.predict({'input1': img})
#     print tmp.output1
    y.append(tmp['output1'])
#   y = model.predict_on_batch(images)
  emb_array[start_index:end_index,:] = y
#   print('y', y)
#   print('emb', emb_array[start_index:end_index,:])
  t1 = time.time()
  
  print('batch: ', i, ' time: ', t1-t0)

from sklearn import metrics
from scipy.optimize import brentq
from scipy import interpolate

tpr, fpr, accuracy, val, val_std, far = lfw.evaluate(emb_array, 
                actual_issame, nrof_folds=lfw_nrof_folds)

print('Accuracy: %1.3f+-%1.3f' % (np.mean(accuracy), np.std(accuracy)))
print('Validation rate: %2.5f+-%2.5f @ FAR=%2.5f' % (val, val_std, far))
auc = metrics.auc(fpr, tpr)
print('Area Under Curve (AUC): %1.3f' % auc)

eer = brentq(lambda x: 1. - x - interpolate.interp1d(fpr, tpr)(x), 0., 1.)
print('Equal Error Rate (EER): %1.3f' % eer)

Skipped 45 image pairs
('batch: ', 0, ' time: ', 4.055538892745972)
('batch: ', 1, ' time: ', 3.6364188194274902)
('batch: ', 2, ' time: ', 3.877336025238037)
('batch: ', 3, ' time: ', 4.0693418979644775)
('batch: ', 4, ' time: ', 4.204995155334473)
('batch: ', 5, ' time: ', 3.9543509483337402)
('batch: ', 6, ' time: ', 3.8213319778442383)
('batch: ', 7, ' time: ', 4.078046083450317)
('batch: ', 8, ' time: ', 4.504278182983398)
('batch: ', 9, ' time: ', 6.106820821762085)
('batch: ', 10, ' time: ', 5.799245119094849)
('batch: ', 11, ' time: ', 7.1995768547058105)
('batch: ', 12, ' time: ', 5.602373123168945)
('batch: ', 13, ' time: ', 4.0945539474487305)
('batch: ', 14, ' time: ', 4.944597959518433)
('batch: ', 15, ' time: ', 3.806425094604492)
('batch: ', 16, ' time: ', 3.9821810722351074)
('batch: ', 17, ' time: ', 3.879357099533081)
('batch: ', 18, ' time: ', 4.34814715385437)
('batch: ', 19, ' time: ', 4.346083164215088)
('batch: ', 20, ' time: ', 4.0362629890441895)
('batch: ', 21

In [21]:
coreml_model.save('./model/OpenFace.mlmodel')